In [62]:
!pip install stemming

In [2]:
import pandas as pd
import ast 
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import spacy
import en_core_web_sm
import string
from normalise import normalise
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import numpy as np
import multiprocessing as mp
from nltk.tokenize import word_tokenize
from sklearn.base import TransformerMixin, BaseEstimator

In [3]:
file = pd.read_csv('resume_data.csv')

In [4]:
file = file[['Resume_title',  'Description', 'work_experiences',
       'Educations', 
       'Additional Information']]

file.dropna(inplace=True)

In [5]:
file.head()

,Resume_title,Description,work_experiences,Educations,Additional Information
0,Java Developer,"To prove myself dedicated, worthy and energeti...","{0: [{'wtitle:': 'Java Developer'}, {'wcompany...","{0: [{'e_title:': ""Bachelor's in Bachelor of C...",\nTechnical Expertise \n• Operating Systems: W...
1,Software Developer,Working as Software Developer at IngroInfo Sof...,"{0: [{'wtitle:': 'JAVA DEVELOPER'}, {'wcompany...",{0: [{'e_title:': 'MCA in Master of Computer A...,NONE
2,Java developer,Looking for a challenging career which demands...,"{0: [{'wtitle:': 'Java Developer'}, {'wcompany...","{0: [{'e_title:': ""Bachelor's in Electrical En...","\nTECHNICAL SKILLS \n \nFrameworks: Spring, Sp..."
3,Seeking innovative and challenging career assi...,NONE,"{0: [{'wtitle:': 'Java Developer'}, {'wcompany...","{0: [{'e_title:': 'BE in Computer'}, {'e_schoo...",NONE
4,NONE,NONE,"{0: [{'wtitle:': 'Java Developer'}, {'wcompany...",{0: [{'e_title:': 'Bachelor of Engineering in ...,NONE


In [9]:
def clean_work_exp_column(content):
    
    text = '''
    '''
    
    try:
        res = ast.literal_eval(content)
        wtitle = set()
        wdescr = set()
        for tup in res:
            wtitle.add(list(res[tup][0].values())[0] + ' ')
            desc = list(res[tup][5].values())[0]
            desc = re.sub(r'[^\x00-\x7F]+',' ', desc)
            if desc.upper() == 'NONE':
                continue
            wdescr.add(desc + ' ')

        for i in wtitle:
            text += i + ' '

        for i in wdescr:
            text += i


    except Exception as e:
        print(e)
    return text

In [13]:
print(clean_work_exp_column(file['work_experiences'][3]))


    Java Developer    Working as a Java Developer at Atos Syntel. 
  Being a part of team to work on US based project in Logistics Domain to improve business operations by enhancing existing business software functionalities.  


In [14]:
def clean_education_column(content):
    
    text = '''
    '''
    
 
    try:
        res = ast.literal_eval(content)
        for tup in res:
                title = list(res[tup][0].values())[0]
                title = re.sub(r'[^\x00-\x7F]+',' ', title)
                if title.upper() == 'NONE':
                    continue
                text += title
    except Exception as e:
        print(e)
    return text





In [18]:
print(clean_education_column(file['Educations'][9]))


    BTECH in IT in Nanded Diploma in IT 


In [75]:
def clean_title_column(content):
    
    text = '''
    '''
    
  
    try:
        if content.upper() == 'NONE':
            return ''
        text += content
    except Exception as e:
        print(e)
    return text


In [23]:
print(clean_title_column(file['Resume_title'][8]))


    mca / with 3 years of development experience 


In [25]:
def clean_desc_column(desc):
    
    text = '''
    '''
    
    try:
        if desc.upper() == 'NONE':
            return ''

        desc = re.sub(r'[^\x00-\x7F]+',' ', desc)
        text += desc
    except Exception as e:
        print(desc)
    return text


In [27]:
print(clean_desc_column(file['Description'][6]))


    To secure a challenging position where I can effectively contribute my skills as Software Professional, possessing competent 
Technical Skills. 


In [28]:
def clean_additonal_column(info):
    
    text = '''
    '''
    
    try:
        if info.upper() == 'NONE':
            return ''

        info = re.sub(r'[^\x00-\x7F]+',' ', info)
        text += info
    except Exception as e:
        print(info)
    return text




In [29]:
print(clean_additonal_column(file['Additional Information'][0]))


    
Technical Expertise 
  Operating Systems: Windows 7/8/10. 
  Languages Known: JAVA, C, C++, HTML, CSS, Bootstrap, J2EE, JSP and JavaScript. 
  Frameworks Hibernate and Spring. 
  Packages: MYSQL 
Development Tools: Eclipse and Netbeans.


In [86]:
import spacy
from stemming.porter2 import stem
sp = spacy.load('en_core_web_sm')

all_stopwords = sp.Defaults.stop_words


def preprocess_text(text):
    
    #print('processing')
    
    text = text.lower()
    text = re.sub(r'[^\x00-\x7F]+',' ', text)
    text = re.sub(r'[^a-zA-Z0-9 ]', '', text).strip()
    text = re.sub(r'\w*\d\w*', '', text).strip()
    text = re.sub(' +', ' ', text)
    
    pure_words_list = remove_stops(text)
    stemmed_text = stem_all_words(pure_words_list)
    
    return stemmed_text


def remove_stops(text):
    
    text_tokens = word_tokenize(text)
    tokens_without_sw= [word for word in text_tokens if not word in all_stopwords]

    return tokens_without_sw

def stem_all_words(text):
    
    w = [stem(word) for word in text]
    return ' '.join(w)



In [87]:
dataset = []

In [88]:
for index, row in file.iterrows():
    val = [clean_title_column(row['Resume_title']).strip(),
           clean_desc_column(row['Description']).strip(),
           clean_education_column(row['Educations']).strip(),
           clean_work_exp_column(row['work_experiences']).strip(),
           clean_additonal_column(row['Additional Information']).strip()]
    
    val = ' '.join(val)
    #print(val)
    val = preprocess_text(val)
    
    #print(val)
    dataset.append(val)
    
    

unexpected indent (<unknown>, line 1)
unexpected indent (<unknown>, line 1)
unexpected indent (<unknown>, line 1)
unexpected indent (<unknown>, line 1)


In [89]:
print('Done')

Done


In [90]:
len(dataset)

14798

In [91]:
import pickle
with open('dataset_processed.pkl', 'wb') as f:
    pickle.dump(dataset, f)